In [ ]:
# Pylake package that LUMICKS provides
import lumicks.pylake as pylake

# Python module for more elaborate mathematical operations
import numpy as np

# Plotting module
import matplotlib.pyplot as plt

# Module we use for looking for files in a folder
import glob

# Fitting module
import lmfit
%matplotlib inline

# Fluorescence Recovery After Photobleaching (FRAP)
This notebook will demonstrate how to do some basic curve fitting to FRAP data acquired with bluelake.

In a typical FRAP experiment a selected region will be photobleached by an intense laser pulse. After
bleaching, the fluorescence will typically recover. The recovery of this intensity contains information
on the influx of molecules that have not been photobleached.

In this experiment, we are dealing with protein droplets. We subsequently photobleach these droplets by
using an intense laser pulse. After this initial bleaching phase, the laser is returned to a low power
and we can observe the photobleached proteins in the droplets exchange with the fluorescent proteins
outside of the droplet.

First step will be to look at which h5 files we have in the folder.

In [ ]:
# Get list of all the Marker .h5 files in the folder
files = glob.glob('*.h5')
print("Found {} files in folder".format(len(files)))

for file in files:
    print('\''+ file + '\',')

# Loading Bluelake HDF5 data

To load the data corresponding to this experiment, we call the filename with `pylake.File`. This will open the file for us,
and allow us to interact with its contents.

In [ ]:
# We want to use the first file
file_index = 0

# Load in the file
file = pylake.File(files[file_index])

# Obtain the Green fluorescence channel and downsample the data to 100 Hz.

Bluelake typically acquires data at very high frequencies. While for some applications this is crucial, in our case, we could perform our analysis quicker if we were to integrate photons over a larger time frame. Therefore, we wish to downsample the raw photon counts to a lower sample rate.

Let's grab the green fluorescence channel and downsample the data to 100 Hz. To do this, we can use the
information stored in the field `sample_rate` (to get the current sample rate) in conjunction with the
function `downsampled_by` which can help us downsample the data. Considering that these are photon counts,
we would like to sum them over the new time window when downsampling instead of averaging (which is the default downsampling method). Hence, for the reduce function
we provide `np.sum` to indicate that the samples have to be summed over.

In [ ]:
green_channel = file['Photon count']['Green']
desired_sample_rate = 100
downsampled = green_channel.downsampled_by(int(green_channel.sample_rate / desired_sample_rate), reduce=np.sum)

Let's plot the data by calling `plot` on the downsampled channel data:

In [ ]:
# Plot the full time trace
plt.figure(figsize=(20,5))
downsampled.plot()
plt.axvline((4.3), color='red')
plt.axvline((20.7), color='black')
plt.ylabel('Green Channel Photon Counts')
plt.legend(['Photon Counts', 'Turn confocal laser to high power','Turn confocal laser down to lower power'])

# Plot the recovery curve
A clear spike in photon counts can be observed after increasing the laser power. The laser was kept on for 15 seconds,
after which it was turned back to low power to observe the recovery curve.

Let's slice the region between 20.8 and 80 seconds from our dataset and plot it.

In [ ]:
# Plot the region of interest
plt.figure(figsize=(20,5))
green_counts = downsampled['20.8s':'80s']
green_counts.plot()
plt.ylabel('Green Channel Photon Counts')

# Fit the recovery curve
We would like to know how quickly the fluorescence recovers and to get an indication of the time scale at which this happens, we can fit a simple mono-exponential model to this recovery:

$G = G_0 + \delta G \left(1-e^{-t/\tau}\right)$

Here $G_0$ refers to the fluorescence level immediately after bleaching, and $\delta G$ refers to the total fluorescence recovery. We use the Python module `lmfit` to make this easier.

In [ ]:
# Define function that we want to fit
def frap(t, tau, dG, G_0):
    return dG * (1 - np.exp(-t / tau) ) + G_0

# Get time traces in seconds
time = green_counts.timestamps
time = (time - time[0]) * 1e-9
raw_data = green_counts.data

# Here we fit the data to the defined model
model = lmfit.Model(frap)
params = model.make_params(tau=1, dG=np.max(raw_data)-np.min(raw_data), G_0=np.min(raw_data))
fitres = model.fit(raw_data, t=time, params=params, method='least_squares')

# output the result of the fit
fitres

In [ ]:
plt.figure(figsize = (9,5))
green_counts.plot()
plt.plot(time, fitres.best_fit)
plt.ylabel('Green Channel Photon Count')
plt.xlabel('Time (s)')

plt.title(fr'$G(t) = G_0 + \delta G (1-e^{{-t/\tau}})$, with tau = {fitres.params["tau"].value}');